In [1]:
!git clone https://github.com/routsourav1729/CUB_200_2011.git

Cloning into 'CUB_200_2011'...
remote: Enumerating objects: 12201, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12201 (delta 2), reused 9 (delta 1), pack-reused 12191
Receiving objects: 100% (12201/12201), 1.05 GiB | 24.20 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (11792/11792), done.


In [ ]:
!pip install efficientnet-pytorch

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt



# Dataset preparation with training and validation split


def get_data_loaders(data_dir, batch_size=32, num_workers=0, valid_size=0.1):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    dataset = datasets.ImageFolder(data_dir, transform=transform)

    # Splitting dataset into training and validation
    num_train = len(dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(
        dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
    val_loader = DataLoader(dataset, batch_size=batch_size,
                            sampler=valid_sampler, num_workers=num_workers)

    return train_loader, val_loader

# Customized EfficientNet model with ImageNet pretrained weights


class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientNet, self).__init__()
        self.base_model = EfficientNet.from_pretrained(
            'efficientnet-b0')  # Pretrained on ImageNet
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.base_model._fc.in_features, num_classes)

    def forward(self, x):
        x = self.base_model.extract_features(x)
        x = self.base_model._avg_pooling(x)
        x = x.flatten(start_dim=1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Function to train the model and plot training loss and accuracy


def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.0001, weight_decay=1e-5, checkpoint_path='checkpoint.pth'):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    train_losses, val_accuracies = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_accuracy = 100 * correct / total
        val_accuracies.append(epoch_accuracy)

        print(
            f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}, Accuracy on validation set: {epoch_accuracy}%")

        # Save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }, '/content/CUB_200_2011/model_st.pth')

    # Plotting training loss and validation accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Validation Accuracy')
    plt.legend()

    plt.show()


# Assuming the path to your dataset and initializing parameters
data_dir = '/content/CUB_200_2011/train'
train_loader, val_loader = get_data_loaders(data_dir)
# Assuming 200 classes in CUB-200-2011
model = CustomEfficientNet(num_classes=200)

# Train the model and plot curves
train_model(model, train_loader, val_loader)

ModuleNotFoundError: No module named 'efficientnet_pytorch'

In [ ]:


# Customized EfficientNet model


# Function to load the test dataset


def load_test_data(data_dir, batch_size=32, num_workers=0):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225]),
    ])

    test_dataset = datasets.ImageFolder(data_dir, transform=transform)
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return test_loader

# Function to evaluate the model on the test set


def evaluate_model(model, test_loader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%')
    return accuracy


# Load the saved model
checkpoint_path = '/content/CUB_200_2011/model_st.pth'
num_classes = 200  # Assuming 200 classes in CUB-200-2011
model = CustomEfficientNet(num_classes=num_classes)

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])

# Load test data
test_data_dir = '/content/CUB_200_2011/test'
test_loader = load_test_data(test_data_dir)

# Evaluate the model
test_accuracy = evaluate_model(model, test_loader)

# Optionally, plot the test accuracy (here, just showing the value)
plt.figure(figsize=(5, 3))
plt.bar(['Test Accuracy'], [test_accuracy])
plt.ylabel('Accuracy (%)')
plt.title('Model Test Accuracy')
plt.show()

AssertionError: Missing keys when loading pretrained weights: ['_fc.weight', '_fc.bias']

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt

# Assuming CUB-200-2011 has 200 classes
num_classes = 200
batch_size = 32
num_epochs = 20
learning_rate = 0.0005

# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=299, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=299),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),

    'test':
    transforms.Compose([
        transforms.Resize(size=299),
        transforms.CenterCrop(size=299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# Datasets from folders
data = {
    'train':
    datasets.ImageFolder(root='/content/CUB_200_2011/train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='/content/CUB_200_2011/test', transform=image_transforms['test']),
}

# Dataloader iterators, make sure to shuffle

train_loader = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(data['test'], batch_size=batch_size, shuffle=False)




In [ ]:





class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientNet, self).__init__()
        self.base_model = EfficientNet.from_pretrained(
            'efficientnet-b2')  # Pretrained on ImageNet
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.base_model._fc.in_features, num_classes)

    def forward(self, x):
        x = self.base_model.extract_features(x)
        x = self.base_model._avg_pooling(x)
        x = x.flatten(start_dim=1)
        x = self.dropout(x)
        x = self.fc(x)
        return x



In [ ]:
model = CustomEfficientNet(num_classes=200)

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth
100%|██████████| 35.1M/35.1M [00:00<00:00, 258MB/s]


Loaded pretrained weights for efficientnet-b2
9,391,794 total parameters.
9,391,794 training parameters.


In [ ]:


# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.2f}%')


print('Finished Training')

# Testing the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')


Epoch [1/20], Loss: 3.7303, Train Accuracy: 23.39%
Epoch [2/20], Loss: 1.5202, Train Accuracy: 61.53%
Epoch [3/20], Loss: 0.8815, Train Accuracy: 76.18%
Epoch [4/20], Loss: 0.5914, Train Accuracy: 84.00%
Epoch [5/20], Loss: 0.4215, Train Accuracy: 88.71%
